<a href="https://colab.research.google.com/github/nifemi-alonge/weather_api/blob/main/wind_speed_visual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualisation of wind speeds along US East Coast



In [ ]:
#@title Packages

# import packages
import requests
import json
import folium
import pandas as pd
import folium.plugins
import branca
import branca.colormap as cm
from branca.element import Figure

In [ ]:
#@title API Call
# coords of locations to pull
coords = [("43.69920671939866", "-70.25652067132195"), ("43.09177918858237", "-70.77030519089477"), ("42.53016640811762", "-70.89532776688091"),
          ("42.36465506707998", "-71.04968585557586"), ("41.72383011727988", "-69.93818993017744"), ("41.04454411906492", "-71.9566052032721"),
          ("40.789838709583485", "-74.0050555570847"), ("40.072732968736084", "-75.3045138517396"), ("39.28935369773514", "-74.38406422635904"),
          ("39.343729351386266", "-76.5745281051535"), ("38.95931011023964", "-77.04469612568182"), ("37.975003170906945", "-75.33778178189938"),
          ("37.558303999252836", "-77.46065764040752"), ("36.853562182282616", "-75.96773952621893"), ("34.270436464509174", "-77.9508275073204"),
          ("33.69503607015078", "-78.85484991443307"), ("32.7670370309246", "-79.99565265045386"), ("32.08300765231105", "-81.08821844864184"),
          ("31.154700025303736", "-81.4717018092687"), ("30.450906634772494", "-81.60670301239594"), ("29.926278993977878", "-81.27077084063998"),
          ("29.22707191643628", "-81.02501414056181"), ("26.88903285720925", "-80.0893031123427"), ("26.26923584642122", "-80.15427602707545"),
          ("25.889912127885143", "-80.0893031123427"), ("28.001616224214235", "-80.54113001973084"), ("30.36618093952354", "-81.8298461120684"),
          ("35.95708001164725", "-75.61832134116356"), ("44.82359030893502", "-68.83190766887827"), ("44.67307534633069", "-63.5781403078735"),
          ("45.53287192804436", "-61.002903812720206"), ("47.11071164383547", "-56.1160096977717"), ("45.61888680352545", "-61.17387386961969"),
          ("28.329412770364446", "-80.68440642705451"), ("35.42199846265172", "-75.48941659768687"), ("43.86050969850932", "-66.11172123906215"),
          ("45.31954984238485", "-66.10622807507801"), ("43.98711832639409", "-64.71096442310555"), ("45.025657444700244", "-62.03653278381564"),
          ("34.73335671287728", "-76.73054763577792"), ("47.61035466486121", "-52.44484509807228"), ("46.09470645692802", "-60.1757240860617"),
          ("46.78385658815272", "-53.84311850306151")
          ]

api_key = "f8ac22f77d262ceae9f13650deabc926"
us_data = []

# run api call thorugh coords via loop
for lat,lon in coords:
  url = "https://api.openweathermap.org/data/2.5/weather?lat=%s&lon=%s&appid=%s&units=metric" % (lat, lon, api_key)
  response = requests.get(url)
  data = json.loads(response.text)
  us_data.append(data)
  #print(us_data)


In [ ]:
#@title Convert list of dictionaries to data frame

# save df
df = pd.DataFrame(us_data)
df.head()

In [ ]:
#@title Extract relevant values as new columns

# extract into new columns
df['lat'] = pd.DataFrame([x for x in df['coord']])['lat']
df['lon'] = pd.DataFrame([x for x in df['coord']])['lon']
df['speed'] = pd.DataFrame([x for x in df['wind']])['speed']

# display
df.head()

In [ ]:
#@title Subset dataframe to relevant columns only

# subset dataframe
df = df[['id', 'name', 'lat', 'lon', 'speed']]
# display
df.head()

In [ ]:
#@title Create visualisation

# create a copy of the data
data = df
# set colour map
colormap = cm.LinearColormap(colors=['yellow', 'darkred'], index=[min(data.speed),max(data.speed)],vmin=min(data.speed),vmax=max(data.speed))
# create folium map object
map = folium.Map(location=[40.730610, -73.935242],  tiles="Stamen Toner", zoom_start=5)
# create feature group
fg = folium.FeatureGroup(name="Wind Speed along North American East Coast in m/s")

# for loop
# create folium map with circle markers
for i in range(len(data)):
    folium.CircleMarker(
        location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
        radius=25,
        fill_color=colormap(data.iloc[i]['speed']),
        color=None,
        fill_opacity=0.5,
        popup = "Wind Speed in {0} is %sm/s".format(data.iloc[i]['name']) %data.iloc[i]['speed']
    ).add_to(map)

# add circle markers to map
#map.add_child(colormap)

# add colour bar caption
colormap.caption = 'Wind Speed along US Eastern Coast in m/s'
colormap.add_to(map)

# display map
display(map)

# Discussion

## Overview

The visual above shows the locations (circle markers) and the scale of possible storm disruption occurring on the Eastern Coast of North America.

The scale runs from yellow (low wind speed where wind speed is measured in m/s) to dark red (high wind speed).

As of 10th February the data shows high wind speeds along the East coast of Canada, New Jersey and Virginia in the US. Whilst around New Hampshire and the Carolinas in the US are experiencing low to medium wind speeds.

As of 15th February data shows very high wind speeds in Canadian Island of Newfoundland and calm winds across the rest of the US Eastern Coast.  

## Data

To start a project linking extreme weather to potential supply chain disruption in the UK you would firstly need to think about data.

- This includes what data is a storm proxy (wind speed, precipitation, visibility etc...)
- The granularity of the data required (by city, state, country level etc...)
- Data access (sources and their trustworthiness - e.g. Met Office)
- Data frequency (how often does MO publish vs. using an API)

The next thing would be to get a view of current major UK suppliers - what important goods (e.g. medicines, electronic hardware, PPE etc...) are coming from what regions. I imagine this is data departments such as Business and Trade hold.

## Method

I would undertake an exercise to match geospatial weather data to the goods the UK is importing. For instance I would take the approach of automating an API call that would gather the relevant weather data for the regions we are interested in - a list of regions could be saved in a csv, easily updated and the API call would run through the coordinates/place names in the list. Also contained in the csv would be a column indicating what good we procured from that region (e.g. Taiwan and microchips) and also include in the function an argument to retain that information. That way I would produce a dataframe pulling in the weather data and the information necessary to plot it and also what goods are relevant to that region. Producing a map similar to above it is also very easy to include the good information in the popup. Therefore you could create a global map showing supply chain locations with weather data highlighted using an appropriate scale and can easily see what goods are affected. The goods could also be used as an argument in the analysis and different goods can be plotted in different colours.

## Presentation

The data would be presented using an interactive map (static depending on the customer's requirements) highlighting the scale (using colour gradients) of certain weather conditions and the goods impacted - this could be through multiple graphs for categories of goods or 1 graph containing the goods information as a popup or different good categories coloured using different scales.  

This type of analysis would give us visibility of the regions vital to UK supply chains, what the current weather conditions are and what goods would be affected.

## Risks

This information would show you the risks to certain goods from weather conditions - especially if you tracked weather patterns as they hit coast lines. For instance the map shows (as of 10th Feb) regions of the mid atlantic being affected by strong winds - this could have an impact on the shipping out of Baltimore Harbour. Port of Baltimore has vast amounts of steel, lumber, forest products passing through and as almost 60% of the UK's wood pellet comes from the USA with the other 21% coming from Canada - this could cause disruption. For instance (as of 15th Feb) Newfoundland in Canada is experiencing extreme wind speeds of 22m/s. Port of St John's is a key port on the island and they are big exporters of Petroleum products and fish products. The UK is a key importer of Newfoundland shellfish (we imported $54mn worth of seafood in 2021) therefore there could be disruption in this market.  

Disruption could be caused by rougher seas from shipping and potentially large lorries overturning on the way to the ports.

Extreme weather can also cause power failures and complete shut downs like we saw with the storms at the end of 2022 that particularly impacted the city of Buffalo and the wider New York state.  

There are risks that approaching weather patterns sometimes do not strike land or become much more mild - this could lead to an overreaction and potentially to the UK seeking an unecessary and more expensive alternative supply if the data is acted upon too soon.

Need to take into account different country's preparedness levels - e.g. snow in canada - drivers prepared with snow tyres etc... so snow may have less impact on Canadian exporters than British.

Risk that the algorithm OpenWeather uses to predict the weather is not well trained / not publicly available - unable to verify the accuracy of their weather information. Alternative could be Met Office API which publishes verification methods, algorithm revision timelines so a more trusted source.  

## Benefits

This would bring are greater regional and geographic understanding of weather impacts to supply chains - the ability to track and monitor emerging weather patterns essentially allows us to predict what goods are likely to face disruption. This allows us to put in place contingencies and potentially secure alternative suppliers of key goods.

It allows for a granular view as we could choose to monitor specific locations only.

This API sources data from multiple data sources - cross checking the information and using their own algorithm to predict the weather based on this information.
  

# References

https://openweathermap.org/api

https://stackoverflow.com/questions/35711059/extract-dictionary-value-from-column-in-data-frame

https://gis.stackexchange.com/questions/257791/fill-color-not-showing-appropriately-in-folium

https://stackoverflow.com/questions/51537748/unable-to-change-default-radius-value-using-folium-circlemarker


https://stackoverflow.com/questions/2960772/how-do-i-put-a-variable-s-value-inside-a-string-interpolate-it-into-the-string

https://stackoverflow.com/questions/37466683/create-a-legend-on-a-folium-map

https://medium.com/@kusohhan/mapping-data-using-folium-5f00cfb6b2cf